## Make Calls to Yahoo Finance ##

In [2]:
# libraries
import pandas as pd
import yfinance as yf


In [6]:
# test yf
data = yf.Ticker(["PLTR", "AAPL"])
df = data.history(period="3mo")
print(df.head())

AttributeError: 'list' object has no attribute 'upper'

In [4]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-12-23 00:00:00-05:00,80.650002,82.019997,77.410004,80.690002,93819500,0.0,0.0
2024-12-24 00:00:00-05:00,81.089996,84.796997,81.000000,82.379997,64983300,0.0,0.0
2024-12-26 00:00:00-05:00,83.160004,83.489998,81.300003,82.139999,60377100,0.0,0.0
2024-12-27 00:00:00-05:00,81.419998,82.000000,77.930000,79.080002,66975900,0.0,0.0
2024-12-30 00:00:00-05:00,77.040001,79.239998,76.110001,77.180000,60850900,0.0,0.0
2024-12-31 00:00:00-05:00,77.580002,77.980003,73.650002,75.629997,56267900,0.0,0.0
2025-01-02 00:00:00-05:00,76.199997,76.529999,72.419998,75.190002,72217200,0.0,0.0
2025-01-03 00:00:00-05:00,75.389999,79.980003,75.190002,79.889999,62415000,0.0,0.0
2025-01-06 00:00:00-05:00,78.690002,80.059998,74.610001,75.919998,105619500,0.0,0.0
